In [3]:
# Setup
import os
import numpy as np # For general mathematical operations
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt # For plotting the results
from torchinfo import summary # For model summaries
from torch.utils.tensorboard import SummaryWriter # For writing into tensorboard
import nrrd # For reading and manipulating nrrd files
from glob import glob
import torch.nn.functional as F
%load_ext autoreload
%autoreload 3

# Develop focal loss

Specifically solve the alpha issue

In [4]:
alpha = torch.Tensor(
[0.5, 1.0, 4.0, 1.0, 4.0, 4.0, 1.0, 1.0, 3.0, 3.0]
)  # TODO: focal loss weights per channels from the paper


gamma = 2.0
dims = [2, 10, 20, 30, 30]
weights = torch.ones(dims)
alpha_transformed = (weights.transpose(1,-1)*alpha).transpose(1,-1).view(-1)

targets = torch.rand(dims)
inputs = (torch.rand(dims) + targets) / 2

# Add error specifically to channel 0
inputs[0,...] = torch.rand(dims[1:])

orig_input_shape = inputs.shape
# flatten label and prediction tensors
inputs = inputs.view(-1)
targets = targets.view(-1)
# first compute binary cross-entropy

BCE = F.binary_cross_entropy(inputs, targets, weight=alpha_transformed, reduction="mean")

BCE_EXP = torch.exp(-BCE)
focal_loss = (1 - BCE_EXP) ** gamma * BCE


focal_loss


tensor(1.2587)

# Then the dice loss


In [390]:
dims = [2,10, 50, 100, 50]
def old(inputs, targets):
    channels = inputs.size()[1]
    inputs = inputs[:].contiguous().view(-1)
    targets = targets[:].contiguous().view(-1)
    intersection = (inputs * targets).sum()
    dice = ((2.0 * intersection) / (inputs.sum() + targets.sum())) / channels
    return dice

def new(inputs, targets, return_per_channel=False):
    # Compute the elementwise operations p * y and p + y
    dice_top = 2 * inputs * targets + 1e-4
    dice_bottom = (inputs + targets + 1e-4)
    dice = dice_top / dice_bottom
    dsc_per_channel = dice.mean(dim=(0,3,2,4))
    dsc_avg = dsc_per_channel.mean()

    if return_per_channel:
        return dsc_avg, dsc_per_channel

    return dsc_avg

targets = (torch.rand(dims))
targets[:,1] *= 10 # Big organ -> 3 times the size
targets[:,0] *= 1 # Small organ -> Half the size
targets = targets.round(decimals=0)
targets[targets > 1] = 1

inputs = torch.nn.Softmax(dim=1)(torch.rand_like(targets))

good_ch = 1
bad_ch = 2

basic_error = 1.0
good_error =  0.
bad_error = 0.3

diff = targets - inputs
inputs += (1.0-basic_error)*diff # Normal error

diff = targets - inputs
inputs[:,good_ch] +=(1-good_error)*diff[:, good_ch] # Good channel

diff = targets - inputs
inputs[:, bad_ch] += (1.0-bad_error)*diff[:, bad_ch] # BAD Channel

old_dsc = old(inputs, targets)
new_dsc = new(inputs,targets, return_per_channel=True)

print("Organ sizes")
print((targets == 1).sum(dim=(0,2,3,4)) / (targets.numel()/targets.shape[1]))

old_dsc.round(decimals=3), new_dsc

Organ sizes
tensor([0.5003, 0.9503, 0.5008, 0.4999, 0.4997, 0.4994, 0.5001, 0.5014, 0.4985,
        0.4989])


(tensor(0.0450),
 (tensor(0.2151),
  tensor([0.0910, 1.0000, 0.4244, 0.0910, 0.0909, 0.0908, 0.0909, 0.0911, 0.0907,
          0.0907])))

In [363]:
dice_top = 2 * inputs * targets + 1e-4
dice_bottom = (inputs + targets + 1e-4)
dice = dice_top / dice_bottom
dsc_per_channel = dice.mean(dim=(0,3,2,4))
dsc_per_channel = dice.mean()
#dsc_avg = dsc_per_channel.mean()

print(dsc_per_channel)

tensor(0.0969)


In [145]:
diceloss = 0
dims = (18,27,27)
targets = torch.rand(dims)
inputs = (torch.rand(dims) + targets) / 2

p = inputs
y = targets
diceloss += (2*(p*y)/(p+y)).sum()

diceloss /= dims[1]

diceloss, forward(inputs, targets=targets)

(tensor(230.9566), tensor(0.5849))

In [130]:
targets = torch.randn(2, 3).round(decimals=0)
inputs = torch.Tensor([2])


print(inputs)
print(targets)
torch.cross(targets, torch.Tensor([2]), dim=-1)

tensor([2.])
tensor([[-1., -1.,  0.],
        [ 1., -0.,  0.]])


tensor([[-2.,  2.,  0.],
        [-0., -2.,  2.]])